In [16]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [17]:
#Importing data
openpolicing_path="C:/Users/SwetaMankala/Desktop/Assignments/EAI6000/ma_statewide_2020_04_01.csv"

data=pd.read_csv(openpolicing_path,low_memory=False)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3416238 entries, 0 to 3416237
Data columns (total 24 columns):
raw_row_number                int64
date                          object
location                      object
county_name                   object
subject_age                   float64
subject_race                  object
subject_sex                   object
type                          object
arrest_made                   object
citation_issued               object
warning_issued                object
outcome                       object
contraband_found              object
contraband_drugs              object
contraband_weapons            object
contraband_alcohol            bool
contraband_other              object
frisk_performed               object
search_conducted              bool
search_basis                  object
reason_for_stop               object
vehicle_type                  object
vehicle_registration_state    object
raw_Race                      object
dty

In [19]:
##Convert date to datetype
data.date
data["date"] = pd.to_datetime(data.date, format="%Y-%m-%d")
#Extract time
data['time'] = [d.time() for d in data['date']]
#Convert date back to original form
data['date'] = pd.to_datetime(data['date']).dt.date

In [20]:
data.head(5)

,raw_row_number,date,location,county_name,subject_age,subject_race,subject_sex,type,arrest_made,citation_issued,...,contraband_alcohol,contraband_other,frisk_performed,search_conducted,search_basis,reason_for_stop,vehicle_type,vehicle_registration_state,raw_Race,time
0,1,2007-06-06,MIDDLEBOROUGH,Plymouth County,33.0,white,male,vehicular,False,True,...,False,NaN,NaN,False,NaN,Speed,Passenger,MA,White,00:00:00
1,2,2007-06-07,SEEKONK,Bristol County,36.0,white,male,vehicular,False,False,...,False,False,False,True,other,NaN,Commercial,MA,White,00:00:00
2,3,2007-06-07,MEDFORD,Middlesex County,56.0,white,female,vehicular,False,False,...,False,NaN,NaN,False,NaN,NaN,Passenger,MA,White,00:00:00
3,4,2007-06-07,MEDFORD,Middlesex County,37.0,white,male,vehicular,False,False,...,False,NaN,NaN,False,NaN,NaN,Commercial,MA,White,00:00:00
4,5,2007-06-07,EVERETT,Middlesex County,22.0,hispanic,female,vehicular,False,True,...,False,NaN,NaN,False,NaN,NaN,Commercial,MA,Hispanic,00:00:00


In [21]:
#Convering race, sex, location etc to categorical
data.iloc[:,np.r_[2:4,5:8,9:12,19:23]]=data.iloc[:,np.r_[2:4,5:8,9:12,19:23]].astype("category")

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3416238 entries, 0 to 3416237
Data columns (total 25 columns):
raw_row_number                int64
date                          object
location                      category
county_name                   category
subject_age                   float64
subject_race                  category
subject_sex                   category
type                          category
arrest_made                   object
citation_issued               category
warning_issued                category
outcome                       category
contraband_found              object
contraband_drugs              object
contraband_weapons            object
contraband_alcohol            bool
contraband_other              object
frisk_performed               object
search_conducted              bool
search_basis                  category
reason_for_stop               category
vehicle_type                  category
vehicle_registration_state    category
raw_Race        

In [28]:
df = pd.DataFrame(data)
median1 = df['subject_age'].median()
df['subject_age'].fillna(median1, inplace = True)

In [36]:
df.isnull().sum()

raw_row_number                      0
date                                0
location                         6666
county_name                      6666
subject_age                         0
subject_race                        0
subject_sex                         0
type                                0
arrest_made                       916
citation_issued                   916
warning_issued                    916
outcome                             0
contraband_found              3360508
contraband_drugs              3360508
contraband_weapons            3360508
contraband_alcohol                  0
contraband_other              3360508
frisk_performed               3361607
search_conducted                    0
search_basis                  3365209
reason_for_stop               1659589
vehicle_type                        0
vehicle_registration_state       9814
raw_Race                         1664
time                                0
race                                0
sex         

In [30]:
#Segregate the values based on the categories, remove the nulls and normalize the data column
df['race'] = pd.Series(len(df['subject_race']), index=df.index)
df['race'] = 0

#To assign null values
df.loc[(df['subject_race'] != 'hispanic') | 
           (df['subject_race'] != 'white') |
           (df['subject_race'] != 'black') |
           (df['subject_race'] != 'asian/pacific islander') |
           (df['subject_race'] != 'other') |
           (df['subject_race'].isnull() == True), 'race'] = np.nan

#To assign the categorical values to the dataframe 'race'
df.loc[(df['subject_race'] == 'hispanic') | 
           (df['subject_race'] == 'white') |
           (df['subject_race'] == 'black') |
           (df['subject_race'] == 'other') |
           (df['subject_race'] == 'asian/pacific islander'), 'race'] = df['subject_race']

race_copy = df['race'].copy(deep = True)

# Fill NaN values.
df['race'].fillna(value = 1, inplace = True)

# Obtain values for every race.Axis=0 for rows
race_copy.dropna(axis = 0, inplace = True)
sorted_race = race_copy.value_counts(normalize = True).sort_index()

# Fill one values for individual person with randomly picked from random choice.
df['race'] = df['race'].apply(lambda x: np.random.choice([x for x in sorted_race.index],
                                replace = True, p = sorted_race) if (x == 1) else x).astype(str)

#Normalize=True prints the relative frequency of the values
print("\nFilled NaNs normalized:\n", df['race'].value_counts(normalize = True))

df['subject_race'] = df['race']
df['subject_race'].value_counts()


Filled NaNs normalized:
 white                     0.744557
black                     0.103519
hispanic                  0.099574
asian/pacific islander    0.049110
other                     0.003239
Name: race, dtype: float64


white                     2543585
black                      353647
hispanic                   340170
asian/pacific islander     167770
other                       11066
Name: subject_race, dtype: int64

In [32]:
#Segregate the values based on the categories, remove the nulls and normalize the data column
df['sex'] = pd.Series(len(df['subject_sex']), index = df.index)
df['sex'] = 0

# Randomly stick sex to every user with NaN value.
df.loc[(df['subject_sex'] != 'male') | 
           (df['subject_sex'] != 'female') |
           (df['subject_sex'].isnull() == True), 'sex'] = np.nan
df.loc[(df['subject_sex'] == 'male') | 
           (df['subject_sex'] == 'female'), 'sex'] = df['subject_sex']


# Create a copy to calculate proportions.
sex_copy = df['sex'].copy(deep = True)

# Fill NaN values.
df['sex'].fillna(value = 1, inplace = True)

# Obtain values for every sex.
sex_copy.dropna(axis = 0, inplace = True)
sorted_sex = sex_copy.value_counts(normalize = True).sort_index()

# Fill one values in suspector_sex_rand with randomly picked from random choice.
df['sex'] = df['sex'].apply(lambda x: np.random.choice([x for x in sorted_sex.index],
                                replace = True, p = sorted_sex) if (x == 1) else x).astype(str)
print("Gender proportions after filled NaNs: \n", df['sex'].value_counts(normalize = True))

df['subject_sex'] = df['sex']
df['subject_sex'].value_counts()

Gender proportions after filled NaNs: 
 male      0.694657
female    0.305343
Name: sex, dtype: float64


male      2373113
female    1043125
Name: subject_sex, dtype: int64

In [33]:
#Segregate the values based on the categories, remove the nulls and normalize the data column
df['outcome_v'] = pd.Series(len(df['outcome']), index = df.index)
df['outcome_v'] = 0

# Randomly stick sex to every user with NaN value.
df.loc[(df['outcome'] != 'citation') | 
           (df['outcome'] != 'warning') |
           (df['outcome'] != 'arrest') |
           (df['outcome'].isnull() == True), 'outcome_v'] = np.nan
df.loc[(df['outcome'] != 'citation') | 
           (df['outcome'] != 'warning') |
           (df['outcome'] != 'arrest'), 'outcome_v'] = df['outcome']


# Create a copy to calculate proportions.
outcome_copy = df['outcome_v'].copy(deep = True)

# Fill NaN values.
df['outcome_v'].fillna(value = 1, inplace = True)

outcome_copy.dropna(axis = 0, inplace = True)
sorted_outcome = outcome_copy.value_counts(normalize = True).sort_index()

# Fill one values in suspector_sex_rand with randomly picked from random choice.
df['outcome_v'] = df['outcome_v'].apply(lambda x: np.random.choice([x for x in sorted_outcome.index],
                                replace = True, p = sorted_outcome) if (x == 1) else x).astype(str)
print("Outcome proportions after filled NaNs: \n", df['outcome_v'].value_counts(normalize = True))

df['outcome'] = df['outcome_v']
df['outcome'].value_counts()

Outcome proportions after filled NaNs: 
 citation    0.63683
warning     0.33618
arrest      0.02699
Name: outcome_v, dtype: float64


citation    2175562
warning     1148472
arrest        92204
Name: outcome, dtype: int64

In [34]:
#Segregate the values based on the categories, remove the nulls and normalize the data column
df['vehicle'] = pd.Series(len(df['vehicle_type']), index = df.index)
df['vehicle'] = 0

df.loc[(df['vehicle_type'] != 'Commerical') | 
           (df['vehicle_type'] != 'Passenger') |
           (df['vehicle_type'] != 'Motorcycle') |
           (df['vehicle_type'] != 'Taxi/Livery') |
           (df['vehicle_type'] != 'Trailer') |
           (df['vehicle_type'].isnull() == True), 'vehicle'] = np.nan
df.loc[(df['vehicle_type'] != 'Commerical') | 
           (df['vehicle_type'] != 'Passenger') |
           (df['vehicle_type'] != 'Motorcycle') |
           (df['vehicle_type'] != 'Taxi/Livery') |
           (df['vehicle_type'] != 'Trailer'), 'vehicle'] = df['vehicle_type']


# Create a copy to calculate proportions.
outcome_copy = df['vehicle'].copy(deep = True)

# Fill NaN values.
df['vehicle'].fillna(value = 1, inplace = True)

outcome_copy.dropna(axis = 0, inplace = True)
sorted_outcome = outcome_copy.value_counts(normalize = True).sort_index()

# Fill one values in suspector_sex_rand with randomly picked from random choice.
df['vehicle'] = df['vehicle'].apply(lambda x: np.random.choice([x for x in sorted_outcome.index],
                                replace = True, p = sorted_outcome) if (x == 1) else x).astype(str)
print("Vehicle Type proportions after filled NaNs: \n", df['vehicle'].value_counts(normalize = True))

df['vehicle_type'] = df['vehicle']
df['vehicle_type'].value_counts()

Vehicle Type proportions after filled NaNs: 
 Passenger      9.312887e-01
Commercial     5.692578e-02
Taxi/Livery    5.727353e-03
Motorcycle     3.671290e-03
Trailer        2.386251e-03
66             2.927197e-07
91             2.927197e-07
Name: vehicle, dtype: float64


Passenger      3181504
Commercial      194472
Taxi/Livery      19566
Motorcycle       12542
Trailer           8152
66                   1
91                   1
Name: vehicle_type, dtype: int64

In [152]:
#One hot encoding categorical variables
data_df=pd.get_dummies(df.loc[:,['subject_race','subject_sex','arrest_made','citation_issued','outcome','contraband_found','contraband_drugs','contraband_weapons','contraband_alcohol','contraband_other','search_basis','reason_for_stop', 'vehicle_type']])

In [208]:
new_data=pd.concat([df['subject_race'],data_df],axis=1)
new_data=pd.concat([df['subject_sex'],new_data],axis=1)
new_data=pd.concat([df['arrest_made'],new_data],axis=1)
new_data=pd.concat([df['citation_issued'],new_data],axis=1)
new_data=pd.concat([df['outcome'],new_data],axis=1)
new_data=pd.concat([df['vehicle_type'],new_data],axis=1)
new_data=pd.concat([df['warning_issued'],new_data],axis=1)

In [209]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
new_data['subject_sex']=lb.fit_transform(new_data['subject_sex'])
new_data['arrest_made']=lb.fit_transform(new_data['arrest_made'])
new_data['citation_issued']=lb.fit_transform(new_data['citation_issued'])
new_data['outcome']=lb.fit_transform(new_data['outcome'])
new_data['vehicle_type']=lb.fit_transform(new_data['vehicle_type'])
new_data['warning_issued']=lb.fit_transform(new_data['warning_issued'])

In [210]:
new_data.head(5)

,warning_issued,vehicle_type,outcome,citation_issued,arrest_made,subject_sex,subject_race,contraband_alcohol,subject_race_asian/pacific islander,subject_race_black,...,"reason_for_stop_Speed,ChildRest","reason_for_stop_Speed,SeatBelt","reason_for_stop_Speed,SeatBelt,ChildRest",vehicle_type_66,vehicle_type_91,vehicle_type_Commercial,vehicle_type_Motorcycle,vehicle_type_Passenger,vehicle_type_Taxi/Livery,vehicle_type_Trailer
0,0,4,1,1,0,1,white,False,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,2,2,0,0,1,white,False,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,4,2,0,0,0,white,False,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,2,2,0,0,1,white,False,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,2,1,1,0,0,hispanic,False,0,0,...,0,0,0,0,0,1,0,0,0,0


In [211]:
Y = new_data['subject_race']
X = new_data
X.drop(['subject_race'], axis=1, inplace=True)
print(X.shape)
print(Y.shape)

(3416238, 46)
(3416238,)


In [224]:
#Split the dataset into train and test
from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size = 0.1, random_state = 0)

In [214]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(C=100, solver='liblinear', random_state = 0)
classifier.fit(X_Train, Y_Train)

LogisticRegression(C=100, random_state=0, solver='liblinear')

In [219]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Predicting the train set results
Y_Pred = classifier.predict(X_Train)

# Making the Confusion Matrix 
print(confusion_matrix(Y_Train, Y_Pred))
print(classification_report(Y_Train, Y_Pred))
print("Train accuracy:", accuracy_score(Y_Train, Y_Pred))

[[ 150961       0       0       0       0]
 [      0  318380       0       0       0]
 [      0       0  306190       0       0]
 [      0       0       0    9970       0]
 [      0       0       0       0 2289113]]
                        precision    recall  f1-score   support

asian/pacific islander       1.00      1.00      1.00    150961
                 black       1.00      1.00      1.00    318380
              hispanic       1.00      1.00      1.00    306190
                 other       1.00      1.00      1.00      9970
                 white       1.00      1.00      1.00   2289113

              accuracy                           1.00   3074614
             macro avg       1.00      1.00      1.00   3074614
          weighted avg       1.00      1.00      1.00   3074614

Train accuracy: 1.0


In [234]:
Pred = classifier.predict(X_Test)
print(confusion_matrix(Y_Test, Pred))
print(classification_report(Y_Test, Pred))
print("Train accuracy:", accuracy_score(Y_Test, Pred))

[[ 16809      0      0      0      0]
 [     0  35267      0      0      0]
 [     0      0  33980      0      0]
 [     0      0      0   1096      0]
 [     0      0      0      0 254472]]
                        precision    recall  f1-score   support

asian/pacific islander       1.00      1.00      1.00     16809
                 black       1.00      1.00      1.00     35267
              hispanic       1.00      1.00      1.00     33980
                 other       1.00      1.00      1.00      1096
                 white       1.00      1.00      1.00    254472

              accuracy                           1.00    341624
             macro avg       1.00      1.00      1.00    341624
          weighted avg       1.00      1.00      1.00    341624

Train accuracy: 1.0
